In [233]:
from nltk.tokenize import RegexpTokenizer
import sys
import re
import numpy as np
import pandas as pd
import math
import random

In [234]:
tokenizer = RegexpTokenizer(r'\w+')

In [235]:
def tokenizedInput(inputFileName,tokenize=False):
    cleaned_list=[]
    with open(inputFileName) as f:
        docs = f.readlines()

    for doc in docs:
        raw=None
        if(tokenize==True):
            raw = tokenizer.tokenize(doc)
        else:
            raw=doc
        cleaned_list.append(raw)
    
    return cleaned_list


In [236]:
# x_train=tokenizedInput('Dataset/Clean1/train_text.txt',tokenize=True)


In [237]:
# print(len(x_train))
# print(len(x_train[0]))

In [281]:
class NaiveBayes:
    
    def __init__(self,x,y,laplace_smoother=1):
        self.x=list(x)
        self.y=list(y)
        self.laplace_smoother=laplace_smoother
        self.distinct_x=0
        self.distinct_y=0
        self.instances=len(y)
        self.x_dict={}
        self.vocabulary={}
        self.y_dict={}
        self.revMapy={}
        self.wordsInClass=[]
        self.instancesInClass=[]
    
    def calculateParameters(self):
#         classes=np.unique(self.npy)
        counter=0
        for i in range(self.instances):
            feature_vector=self.x[i]
            feature_class=self.y[i]
            mapped_class=counter
            
            if(feature_class in self.y_dict):
                mapped_class=self.y_dict[feature_class]
            else:
                self.revMapy[counter]=feature_class
                self.y_dict[feature_class]=counter
                self.wordsInClass.append(0)
                self.instancesInClass.append(0)
                counter+=1
            
            self.instancesInClass[mapped_class]+=1
            
            for words in feature_vector:
                if(words in self.vocabulary):
                    pass
                else:
                    self.vocabulary[words]=1
                
                key=(words,mapped_class)
                self.wordsInClass[mapped_class]+=1
                
                if(key in self.x_dict):
                    self.x_dict[key]+=1
                else:
                    self.x_dict[key]=1
            
        self.distinct_x=len(self.vocabulary)
        self.distinct_y=len(self.y_dict)
#         self.printParameters()
    
    def getLogPrior(self,label):
        return math.log(float(self.instancesInClass[label])/self.instances)
    
    def getLogProb(self,attribute,label):
        
        occurences=0
        key=(attribute,label)
        
        if key in self.x_dict:
            occurences=self.x_dict[key]
        
        occurences+=self.laplace_smoother
        
        total_occurences_in_class=self.wordsInClass[label]
        total_occurences_in_class+=self.distinct_x*self.laplace_smoother
        
        return math.log(float(occurences)/total_occurences_in_class)
        
    def getClass(self,x):
        max_log_prob=-1e9
        label=-1
        
        for i in range(self.distinct_y):
            log_prob_x_given_y=0
            
            for attributes in x:
                log_prob_x_given_y+=self.getLogProb(attributes,i)
            
            log_prob_x=log_prob_x_given_y+self.getLogPrior(i)
            
            if(log_prob_x>max_log_prob):
                max_log_prob=log_prob_x
                label=self.revMapy[i]
            
        return label
    
    def ConfusionMatrix(self,y,predy):
#         confusion=[[0]*self.distinct_y]*self.distinct_y
        confusion = [ [0]*self.distinct_y for _ in range(self.distinct_y) ]
        
        tests=len(y)
        
        for i in range(tests):
            confusion[self.y_dict[predy[i]]][self.y_dict[y[i]]]+=1
        
        for i in range(self.distinct_y):
            for j in range(self.distinct_y):
#                 print((i,j),end=' ')                                
                print("%4d",%(confusion[i][j]),end=' ')
            print()
        
        
    def getAccuracy(self,x,y,printConfusionMatrix=False):
        total_tests=len(y)
        passed_tests=0
        prediction_list=[]
        
        for i in range(total_tests):
            xi=x[i]
            yi=y[i]
            
#             if y[i] in self.y_dict:
#                 pass
#             else:
#                 continue
            pred_yi=self.getClass(xi)
            prediction_list.append(pred_yi)
            if(pred_yi==yi):
                passed_tests+=1
        
        if(printConfusionMatrix==True):
            self.ConfusionMatrix(y,pred_yi)
                
        return [prediction_list,(float(passed_tests))/total_tests]
    
    def getAccuracyRandomPredict(self,x,y):
        
        average_over=10
        total_accuracy=0
        
        for i in range(average_over):
            
            total_tests=len(y)
            passed_tests=0

            for i in range(total_tests):
                yi=y[i]

                pred_yi=random.randint(0,self.distinct_y-1)
                pred_yi=self.revMapy[pred_yi]

                if(pred_yi==yi):
                    passed_tests+=1

            total_accuracy+=(float(passed_tests))/total_tests
        
        return total_accuracy/average_over
    
    def getAccuracyMajorityPredictor(self,x,y):
        max_occ=-1
        majority_class=-1
        total_tests=len(y)
        passed_tests=0        
        
        for i in range(self.distinct_y):
            if(max_occ<self.instancesInClass[i]):
                max_occ=self.instancesInClass[i]
                majority_class=i
        
        if(majority_class==-1):
            return 0
        
        majority_class=self.revMapy[majority_class]
        
        for yi in y:
            if(yi==majority_class):
                passed_tests+=1
                
        return (float(passed_tests))/total_tests       

    def printParameters(self):
        print("Vocabulary Size:",self.distinct_x)
        print("Classes:",self.distinct_y)
        print("X_Y's:",len(self.x_dict))
        
        print("Mapped Classes:", self.y_dict)
        
        print("Instances In Mapped class:",end='')
        
        for counts in self.instancesInClass:
            print(counts,end=' ')
        
        print("")
        print("Words In Mapped class:",end='')
        
        for counts in self.wordsInClass:
            print(counts,end=' ')
        
        print("")
        

SyntaxError: invalid syntax (<ipython-input-281-264cd71cee25>, line 102)

In [239]:
x_train=tokenizedInput('Dataset/Clean1/train_text.txt',tokenize=True)
y_train=tokenizedInput('Dataset/Clean1/imdb_train_labels.txt',tokenize=False)
y_train=list(map(int,y_train))

In [240]:
# len(x_train)

In [241]:
# len(y_train)

In [242]:
# type(y_train[0])

In [277]:
NaiveBayesClassifier=NaiveBayes(x=x_train,y=y_train)

In [278]:
NaiveBayesClassifier.calculateParameters()

In [245]:
print (sum(NaiveBayesClassifier.wordsInClass))

5956618


In [249]:
# print (NaiveBayesClassifier.getAccuracy(x_train,y_train))[1]

In [247]:
x_test=tokenizedInput('Dataset/Clean1/test_text.txt',tokenize=True)
y_test=tokenizedInput('Dataset/Clean1/imdb_test_labels.txt',tokenize=False)
y_test=list(map(int,y_test))

In [248]:
[a,b]=(NaiveBayesClassifier.getAccuracy(x_test,y_test,printConfusionMatrix=False))
# print (b)
# print(NaiveBayesClassifier.ConfusionMatrix(a,y_test))

In [ ]:
# print (NaiveBayesClassifier.getAccuracyRandomPredict(x_train,y_train))

In [ ]:
# print (NaiveBayesClassifier.getAccuracyMajorityPredictor(x_test,y_test))

In [212]:
b

0.38468

In [279]:
NaiveBayesClassifier.ConfusionMatrix(y_test,a)

%4d 3127 %4d 1023 %4d 537 %4d 1141 %4d 142 %4d 187 %4d 94 %4d 134 
%4d 558 %4d 715 %4d 517 %4d 466 %4d 119 %4d 227 %4d 54 %4d 63 
%4d 180 %4d 317 %4d 427 %4d 155 %4d 126 %4d 222 %4d 55 %4d 36 
%4d 179 %4d 131 %4d 74 %4d 124 %4d 14 %4d 28 %4d 5 %4d 18 
%4d 45 %4d 63 %4d 80 %4d 23 %4d 233 %4d 218 %4d 187 %4d 155 
%4d 102 %4d 166 %4d 263 %4d 96 %4d 489 %4d 674 %4d 273 %4d 260 
%4d 12 %4d 13 %4d 9 %4d 6 %4d 58 %4d 47 %4d 51 %4d 90 
%4d 796 %4d 422 %4d 400 %4d 333 %4d 1360 %4d 1032 %4d 1583 %4d 4266 


In [251]:
NaiveBayesClassifier.y_dict


{1: 7, 2: 6, 3: 4, 4: 5, 7: 2, 8: 1, 9: 3, 10: 0}